In [1]:
import zipfile

def unzip_file(zip_filename, output_dir):
    with zipfile.ZipFile(zip_filename, 'r') as zipf:
        zipf.extractall(output_dir)
    print(f"Files extracted to {output_dir}")

# Example usage:
unzip_file('/content/Cells.zip', '/content/')

Files extracted to /content/


In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
from tqdm import tqdm

def augment_and_save_images_with_masks(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, total_samples=2000):
    # Define augmentations with resizing to 256x256
    transform = A.Compose([
        #A.Resize(256, 256, always_apply=True),  # Resize both images & masks to 256x256
        A.HorizontalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        #A.RandomCrop(height=256, width=256, p=0.5),  # Crop within 256x256
    ])

    # Get list of image and mask files
    image_files = sorted([f for f in os.listdir(input_image_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    mask_files = sorted([f for f in os.listdir(input_mask_folder) if f.lower().endswith(('.jpeg'))])

    # Ensure image-mask alignment
    if len(image_files) != len(mask_files):
        print("Warning: Mismatch in the number of images and masks!")

    # Create output directories if they don't exist
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_mask_folder, exist_ok=True)

    num_original_images = len(image_files)

    # Determine augmentations per image needed
    num_augs_per_image = max(1, total_samples // num_original_images)

    index = 0
    for img_name, mask_name in tqdm(zip(image_files, mask_files), total=num_original_images):
        image_file = os.path.join(input_image_folder, img_name)
        mask_file = os.path.join(input_mask_folder, mask_name)

        # Load image in RGB
        image = cv2.imread(image_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB

        # Load mask in RGB (preserve colors)
        mask = cv2.imread(mask_file, cv2.IMREAD_UNCHANGED)  # Load mask in original color (not grayscale)
        mask = cv2.cvtColor(mask, cv2.COLOR_BGR2RGB)  # Ensure it's RGB

        # Skip invalid images/masks
        if image is None or mask is None:
            print(f"Skipping: {img_name} (image/mask not found or invalid)")
            continue

        # Resize both to 256x256
        #image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_LINEAR)
        #mask = cv2.resize(mask, (256, 256), interpolation=cv2.INTER_NEAREST)  # Use NEAREST to preserve segmentation colors

        for _ in range(num_augs_per_image):
            augmented = transform(image=image, mask=mask)
            augmented_image = augmented['image']
            augmented_mask = augmented['mask']

            # Ensure valid augmented images/masks
            if augmented_image is None or augmented_mask is None:
                continue

            # Save augmented image and mask
            output_image_file = os.path.join(output_image_folder, f"augmented_{index}.jpg")
            output_mask_file = os.path.join(output_mask_folder, f"augmented_{index}.jpeg")

            cv2.imwrite(output_image_file, cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR))  # Convert back to BGR for OpenCV
            cv2.imwrite(output_mask_file, cv2.cvtColor(augmented_mask, cv2.COLOR_RGB2BGR))  # Ensure mask remains in RGB

            index += 1
            if index >= total_samples:
                break  # Stop when we reach the desired number of samples

        if index >= total_samples:
            break  # Stop if we have reached 2000 samples

if __name__ == "__main__":
    # Define paths
    input_image_folder = "/content/Cells/Train/images/"
    input_mask_folder = "/content/Cells/Train/masks/"
    output_image_folder = "/content/cell_bi/aug_images_bi/"
    output_mask_folder = "/content/cell_bi/aug_masks_bi/"

    # Set total number of augmented image-mask pairs
    total_samples = 5000

    # Run augmentation
    augment_and_save_images_with_masks(input_image_folder, input_mask_folder, output_image_folder, output_mask_folder, total_samples)

  0%|          | 0/30 [00:00<?, ?it/s]

In [ ]:
IMG_ROWS = 256
IMG_COLS = 256

In [ ]:
import cv2
import numpy as np
import imageio
from scipy import ndimage
from glob import glob
import zipfile
#import tensorflow as tf

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../content/"]).decode("utf8"))

In [ ]:
train_img_paths = sorted(glob('../content/cell_bi/aug_images_bi/*.jpg'))
train_mask_paths = sorted(glob('../content/cell_bi/aug_masks_bi/*.jpeg'))

In [ ]:
test_img_paths = sorted(glob('../content/Cells/test/test_images/*.JPG'))
test_mask_paths = sorted(glob('../content/Cells/test/test_masks/*.jpeg'))

In [ ]:
train_imgs = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))
                        for path in train_img_paths])

train_masks = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))
                        for path in train_mask_paths])

train_masks = train_masks.astype(np.float32)
train_masks[train_masks<=127] = 0.
train_masks[train_masks>127] = 1.
train_masks = np.reshape(train_masks, (*train_masks.shape, 1))

In [ ]:
test_imgs = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))
                        for path in test_img_paths])

test_masks = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))
                        for path in test_mask_paths])

test_masks = test_masks.astype(np.float32)
test_masks[test_masks<=127] = 0.
test_masks[test_masks>127] = 1.
test_masks = np.reshape(test_masks, (*test_masks.shape, 1))

# Train Image

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(0, figsize=(200, 200))
fig.add_subplot(1, 2, 1)
plt.imshow(train_imgs[0])
fig.add_subplot(1, 2, 2)
plt.imshow(np.squeeze(train_masks[0]), cmap='gray')

# Test image

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
fig = plt.figure(0, figsize=(20, 20))
fig.add_subplot(1, 2, 1)
plt.imshow(test_imgs[0])
fig.add_subplot(1, 2, 2)
plt.imshow(np.squeeze(test_masks[0]), cmap='gray')

# Model

In [ ]:
!pip3 install -U segmentation-models
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm
import tensorflow as tf
tf.keras.backend.set_image_data_format('channels_last')

In [ ]:
from tensorflow.keras.optimizers import Adam
#create model
# binary segmentation (this parameters are default when you call Unet('resnet34')
model = sm.Unet('vgg19', classes=1, activation='sigmoid')
optimizer = Adam(learning_rate=0.001)
dice_loss = sm.losses.dice_loss
model.compile(optimizer=optimizer, loss=dice_loss, metrics=['accuracy'])


In [ ]:
import segmentation_models as sm
from tensorflow.keras.optimizers import Adam

# Create the basic Unet model (without any backbone)
model = sm.Unet(input_shape=(256, 256, 3), classes=1, activation='sigmoid', encoder_weights=None)

# Define the optimizer
optimizer = Adam(learning_rate=0.001)

# Define the loss function (Dice loss from segmentation_models)
dice_loss = sm.losses.dice_loss

# Compile the model
model.compile(optimizer=optimizer, loss=dice_loss, metrics=['accuracy'])

In [ ]:
import segmentation_models as sm
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

# Create the basic Unet model (without any backbone)
model = sm.Unet(input_shape=(256, 256, 3), classes=1, activation='sigmoid', encoder_weights=None)

# Define the optimizer
optimizer = Adam(learning_rate=0.001)

# Define the loss function (Dice loss from segmentation_models)
dice_loss = sm.losses.dice_loss

# Define the Dice Coefficient metric
def dice_coefficient(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

# Compile the model with Dice Coefficient as a metric
model.compile(optimizer=optimizer, loss=dice_loss, metrics=['accuracy', dice_coefficient])


In [ ]:
model.summary()

## Set the loss function

In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

# Assuming you have defined the 'model', 'train_imgs', and 'train_masks'

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=1000, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    train_imgs[50:],
    train_masks[50:],
    batch_size=64,
    epochs=50,
    validation_data=(
        train_imgs[:1],
        train_masks[:1]
    ),
    callbacks=[early_stopping]  # Add the early stopping callback
)


## Предсказание модели

In [ ]:
# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training accuracy
plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

test_index = 2

# Get the test image and mask
test_image = test_imgs[test_index]
ground_truth_mask = test_masks[test_index]

# Squeeze the ground_truth_mask to remove the extra dimension
ground_truth_mask = ground_truth_mask.squeeze()

# Predict the mask using the trained model
predicted_mask = model.predict(np.expand_dims(test_image, axis=0))[0]

# Plot the test image, ground truth mask, and predicted mask
plt.figure(figsize=(10, 6))

plt.subplot(131)
plt.imshow(test_image)
plt.title('Test Image')

plt.subplot(132)
plt.imshow(ground_truth_mask, cmap='gray')
plt.title('Ground Truth Mask')

plt.subplot(133)
plt.imshow(predicted_mask, cmap='gray')
plt.title('Predicted Mask')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

test_index = 6

# Get the test image and mask
test_image = test_imgs[test_index]
ground_truth_mask = test_masks[test_index]

# Squeeze the ground_truth_mask to remove the extra dimension
ground_truth_mask = ground_truth_mask.squeeze()

# Predict the mask using the trained model
predicted_mask = model.predict(np.expand_dims(test_image, axis=0))[0]

# Plot the test image, ground truth mask, and predicted mask
plt.figure(figsize=(10, 6))

plt.subplot(131)
plt.imshow(test_image)
plt.title('Test Image')

plt.subplot(132)
plt.imshow(ground_truth_mask, cmap='gray')
plt.title('Ground Truth Mask')

plt.subplot(133)
plt.imshow(predicted_mask, cmap='gray')
plt.title('Predicted Mask')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

test_index = 1

# Get the test image and mask
test_image = test_imgs[test_index]
ground_truth_mask = test_masks[test_index]

# Squeeze the ground_truth_mask to remove the extra dimension
ground_truth_mask = ground_truth_mask.squeeze()

# Predict the mask using the trained model
predicted_mask = model.predict(np.expand_dims(test_image, axis=0))[0]

# Plot the test image, ground truth mask, and predicted mask
plt.figure(figsize=(10, 6))

plt.subplot(131)
plt.imshow(test_image)
plt.title('Test Image')

plt.subplot(132)
plt.imshow(ground_truth_mask, cmap='gray')
plt.title('Ground Truth Mask')

plt.subplot(133)
plt.imshow(predicted_mask, cmap='gray')
plt.title('Predicted Mask')

plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


test_index = 1

# Get the test image and mask
test_image = test_imgs[test_index]
ground_truth_mask = test_masks[test_index]

# Predict the mask using the trained model
predicted_mask = model.predict(np.expand_dims(test_image, axis=0))[0]

# Plot the test image, ground truth mask, and predicted mask
plt.figure(figsize=(15, 5))

# Test Image
plt.subplot(131)
plt.imshow(test_image)
plt.title('Test Image')

# Ground Truth Mask - Squeeze it if needed
ground_truth_mask = np.squeeze(ground_truth_mask)  # Remove any (1) dims
plt.subplot(132)
plt.imshow(ground_truth_mask, cmap='gray')
plt.title('Ground Truth Mask')

# Predicted Mask - Squeeze it if needed
predicted_mask = np.squeeze(predicted_mask)  # Remove any (1) dims
plt.subplot(133)
plt.imshow(predicted_mask, cmap='gray')
plt.title('Predicted Mask')

plt.show()


In [ ]:


import tensorflow as tf
from tensorflow.keras.models import save_model

# Assuming you have defined and trained the 'model'
# model = ...

# Save the trained model to an HDF5 file in your Google Drive
save_model(model, '/content/segmentation_model_unetCELLS.h5')

print("Model saved as 'segmentation_model.h5'")


In [ ]:
test_loss, test_accuracy = model.evaluate(test_imgs, test_masks, batch_size=1)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
train_loss, train_accuracy = model.evaluate(train_imgs, train_masks, batch_size=1)

print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)

In [ ]:
test_paths = sorted(glob('../content/drive/MyDrive/Segmentation/FinalDatasetThesis/val/*.png'))

def test_img_generator(test_paths):
    while True:
        for path in test_paths:
            yield np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))])

pred = model.predict_generator(test_img_generator(test_paths[:10]), len(test_paths[:10]))

## Визуализируем результат

fig = plt.figure(0, figsize=(20, 10))
k = 5
fig.add_subplot(2, 2, 1)
plt.imshow(imageio.imread(test_paths[k]))
fig.add_subplot(2, 2, 2)
plt.imshow(np.squeeze(cv2.resize(pred[k], (TEST_IMG_ROWS, TEST_IMG_COLS))), cmap='gray')
fig.add_subplot(2, 2, 3)
plt.imshow(imageio.imread(test_paths[k+1]))
fig.add_subplot(2, 2, 4)
plt.imshow(np.squeeze(cv2.resize(pred[k+1], (TEST_IMG_ROWS, TEST_IMG_COLS))), cmap='gray')

## Подготавливаем данные для отправки

In [ ]:
def rle_encode(mask):
    pixels = mask.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

imageio.imread(path).shape[0]

In [ ]:
with open('submit.txt', 'w') as dst:
    dst.write('ImageId,EncodedPixels\n')
    for path in test_paths:
        img = np.array([cv2.resize(imageio.imread(path), (IMG_ROWS, IMG_COLS))])
        pred_mask = model.predict(img)[0]
        bin_mask = 255. * cv2.resize(pred_mask, (imageio.imread(path).shape[0], imageio.imread(path).shape[1]))
        bin_mask[bin_mask<=127] = 0
        bin_mask[bin_mask>127] = 1
        rle = rle_encode(bin_mask.astype(np.uint8))
        rle = ' '.join(str(x) for x in rle)
        dst.write('%s,%s\n' % (path.split('/')[-1].split('.')[0], rle))

In [ ]:
import csv

with open('submit.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('submission.csv', 'w') as out_file:
        writer = csv.writer(out_file)
        writer.writerows(lines)